In [ ]:
# List of pipeline steps

**FeatureGenerationPipeline:**
- CustomAttributeAdder(feature_list=['feature1', 'feature2', 'feature3', etc.]):
- RandomAttributeAdder(stratgey=['multiply', 'add', 'substract', 'divide']):

**PreprocessingPipeline:**
- Imputer???
- RandomAttributeTransformer(strategy=['sqaure', 'log', 'sqrt'])
- Balancing(stratgey=['up', 'down', 'other', 'none'])
- Scaling(strategy=['Standard', 'MinMax', 'None'])

**ModelSelectionPipeline:**
- ModelSelector
- CrossValidation


In [62]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer

# classifiers
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# use sklearn pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import SGDClassifier

# custom imports
from funcs import plot_cv_confidence_vs_profit, score_dmc_profit,dmc_profit,cv_preds_and_confusion_matrix
from customClassifiers import CustomModelWithThreshold

# 1. Load Data

In [2]:
df_train = pd.read_csv('train.csv', sep='|')
df_test = pd.read_csv('test.csv', sep='|')

df_train.head()
df_train.tail()
df_train.sample(3)

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
110,4,477,1.36,10,1,1,0.044025,0.002851,0.476190,0
622,6,615,96.76,4,8,3,0.030894,0.157333,0.210526,0
1208,2,810,22.50,9,7,3,0.013580,0.027778,0.818182,0


In [3]:
# split label and predictors
X_train, y_train = df_train.drop(columns=['fraud',]), df_train['fraud']
X_train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111


# 2. Pipeline

### 2.1 FeatureGeneration Pipeline

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [5]:
# scaling class
class Scaling(TransformerMixin):
    
    _strategy = "Standard"
    _scaler = None
    
    def __init__(self, strategy):
        self._strategy = strategy
    
    def fit(self, X, y=None):
        if self._strategy == "Standard":
            self._scaler = StandardScaler()
            return self._scaler.fit(X)
        elif self._strategy == "MinMax":
            self._scaler = MinMaxScaler()
            return self._scaler.fit(X)
        elif self._strategy == "None":
            return self
    
    def transform(self, X):
        
        if self._strategy == "None":
            return X
        else:
            return self._scaler.transform(X)
        

In [6]:
# custom attribute adder
class CustomAttributeAdder(TransformerMixin):
    
    _featureList = ['scannedLineItemsTotal', 'valuePerLineItem', 'quantityModificationsPerLineItem']
    
    def __init__(self, feature_list = ['scannedLineItemsTotal',
                                       'valuePerLineItem',
                                       'quantityModificationsPerLineItem']):
        self.__featurelist = feature_list
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        X['scannedLineItemsTotal'] = X['scannedLineItemsPerSecond'] * X['totalScanTimeInSeconds']
            
        if "valuePerLineItem" in self._featureList:
            X['valuePerLineItem'] = X['grandTotal'] / X['scannedLineItemsTotal']
            
        if "quantityModificationsPerLineItem" in self._featureList:
            X['quantityModificationsPerLineItem'] = X['quantityModifications'] / X['scannedLineItemsTotal']
            
        return X

In [7]:
# custom attribute adder
class RandomAttributeAdder(TransformerMixin):
    
    """This class is still empty and needs to be filled!"""
    
    def __init__(self,):
        pass
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):    
        return X

In [8]:
# featureGeneration pipeline 
# all transform methods should return a pd.DataFrame

featureGeneration_pipeline = Pipeline([
    ("attribs_adder", CustomAttributeAdder()),    # returns pd.dataframe
    ("RandomAttributeAdder", RandomAttributeAdder())
    ])

### 2.2 Preprocessing Pipeline

In [9]:
class Transformer(TransformerMixin):
    
    """This class is still empty and needs to be filled!"""
    
    def __init__(self,):
        pass
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):    
        return X

In [10]:
class Balancing(TransformerMixin):
    
    """This class is still empty and needs to be filled!"""
    
    def __init__(self,):
        pass
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):    
        return X

In [11]:
# scaling class
class Scaling(TransformerMixin):
    
    _strategy = "Standard"
    _scaler = None
    
    def __init__(self, strategy):
        self._strategy = strategy
    
    def fit(self, X, y=None):
        if self._strategy == "Standard":
            self._scaler = StandardScaler()
            return self._scaler.fit(X)
        elif self._strategy == "MinMax":
            self._scaler = MinMaxScaler()
            return self._scaler.fit(X)
        elif self._strategy == "None":
            return self
    
    def transform(self, X):
        
        if self._strategy == "None":
            return X
        else:
            return self._scaler.transform(X)
      

In [12]:
# In this pipeline, we are working with np.array

preprocessing_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),   # This step is actually not needed
    ("transformer", Transformer()),                # This class is still void
    ("balancing", Balancing()),                    # This class is still void
    ("scaler", Scaling(strategy='Standard')),
])

### 2.3 Model Training Pipeline

In [13]:
from sklearn.base import BaseEstimator
from sklearn.linear_model import SGDClassifier

class ClfSwitcher(BaseEstimator):

    def __init__(
        self, 
        estimator = SGDClassifier()
    ):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The classifier
        """ 

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

In [14]:
model_training_pipeline = Pipeline([
    ('feature_generation', featureGeneration_pipeline),
    ('preprocessing', preprocessing_pipeline),
    ('classifier', ClfSwitcher())
])

In [15]:
parameters = [
    {
        'classifier__estimator': [SGDClassifier()],
        'classifier__estimator__penalty': ('l2', 'elasticnet', 'l1'),
        'classifier__estimator__max_iter': [50, 80],
        'classifier__estimator__tol': [1e-4],
        'classifier__estimator__loss': ['hinge', 'log', 'modified_huber']
    },
    {
        'classifier__estimator': [XGBClassifier()],
        'classifier__estimator__n_estimators': [50, 100, 150],
        'classifier__estimator__reg_alpha': [0, 0.05, 0.1]
    },
    {
        'classifier__estimator': [RandomForestClassifier()],
        'classifier__estimator__min_samples_split': [2, 4, 6],
        'classifier__estimator__criterion': ['gini', 'entropy']
    }
]

# 3. Full Pipeline Usage Example

In [19]:
cv = StratifiedKFold(n_splits=10, random_state=42)

def profit_scorer(y, y_pred):
    profit_matrix = {(0,0): 0, (0,1): -5, (1,0): -25, (1,1): 5}
    return sum(profit_matrix[(pred, actual)] for pred, actual in zip(y_pred, y))

profit_scoring = make_scorer(profit_scorer, greater_is_better=True)

In [17]:
gscv = GridSearchCV(model_training_pipeline, parameters, cv=10, n_jobs=-1, scoring=profit_scoring, verbose=3)
gscv.fit(X_train, y_train)

Fitting 10 folds for each of 33 candidates, totalling 330 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done 330 out of 330 | elapsed:   21.7s finished
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('feature_generation', Pipeline(memory=None,
     steps=[('attribs_adder', <__main__.CustomAttributeAdder object at 0x1a1a1c47f0>), ('RandomAttributeAdder', <__main__.RandomAttributeAdder object at 0x1a1a1c46a0>)])), ('preprocessing', Pipeline(memory=None,
     steps=[('imputer', SimpleImpute..._state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'classifier__estimator': [SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=80,
       n_iter=None, n_iter_no_change=5, n_jobs=None, ..._estimator__min_samples_split': [2, 4, 6], 'classifier__estimator__criterion': ['gini', 'entropy']}],
       pre_dispatch='2*n_jobs', refit=

In [18]:
gscv.best_score_

29.49175093134646

In [19]:
gscv.best_params_

{'classifier__estimator': SGDClassifier(alpha=0.0001, average=False, class_weight=None,
        early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
        l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=80,
        n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
        power_t=0.5, random_state=None, shuffle=True, tol=0.0001,
        validation_fraction=0.1, verbose=0, warm_start=False),
 'classifier__estimator__loss': 'hinge',
 'classifier__estimator__max_iter': 80,
 'classifier__estimator__penalty': 'l2',
 'classifier__estimator__tol': 0.0001}

In [20]:
gscv.best_estimator_.named_steps

{'feature_generation': Pipeline(memory=None,
      steps=[('attribs_adder', <__main__.CustomAttributeAdder object at 0x1a1bd606d8>), ('RandomAttributeAdder', <__main__.RandomAttributeAdder object at 0x1065d3c18>)]),
 'preprocessing': Pipeline(memory=None,
      steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
        verbose=0)), ('transformer', <__main__.Transformer object at 0x1065bae80>), ('balancing', <__main__.Balancing object at 0x1065a2518>), ('scaler', <__main__.Scaling object at 0x1065bc1d0>)]),
 'classifier': ClfSwitcher(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
        early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
        l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=80,
        n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
        power_t=0.5, random_state=None, shuffle=True, tol=0.0001,
        validation_fraction=0.1, verbose=0, warm_start=False

# Cross-Validation (training on balanced, validation on unbalanced dataset)

In [16]:
from imblearn.over_sampling import SMOTE

In [31]:
def cross_val_imbalanced(classifier, X, y, sampler):
    kf = StratifiedKFold(n_splits=10, random_state=42)
    cross_val_profit_lst = []
    
    X = pd.DataFrame(X)

    for train_index, test_index in kf.split(X, y):
        # keeping validation set apart and oversampling in each iteration using smote 
        train, test = X.iloc[train_index], X.iloc[test_index]
        target_train, target_test = y.iloc[train_index], y.iloc[test_index]
        X_train_res, y_train_res = sampler.fit_sample(train, target_train.ravel())

        # training the model on oversampled 4 folds of training set
#         classifier.fit(train, target_train)
        classifier.fit(pd.DataFrame(X_train_res), y_train_res)
        # testing on 1 fold of validation set
        test_preds = classifier.predict(test)
        cross_val_profit_lst.append(profit_scorer(target_test, test_preds))
        
#     print ('Cross validated profit: {}'.format(np.sum(cross_val_profit_lst)))
    return np.sum(cross_val_profit_lst)

In [32]:
data_preparation_pipeline = Pipeline([
    ('feature_generation', featureGeneration_pipeline),
    ('preprocessing', preprocessing_pipeline)
])

X_train_prepared = data_preparation_pipeline.fit_transform(X_train)

cross_val_imbalanced(RandomForestClassifier(n_estimators=100, random_state=42), 
                     X_train_prepared, y_train, SMOTE(random_state=42))

-40

# Bayesian Optimization

In [22]:
# Install the library for Bayesian optimization from here: https://github.com/fmfn/BayesianOptimization
from bayes_opt import BayesianOptimization

### Random Forest

In [48]:
data_preparation_pipeline = Pipeline([
    ('feature_generation', featureGeneration_pipeline),
    ('preprocessing', preprocessing_pipeline)
])

In [49]:
X_train_prepared = data_preparation_pipeline.fit_transform(X_train)

In [43]:
def evaluateRandomForest(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    
    model = RandomForestClassifier(n_estimators=int(n_estimators), 
                                   max_depth=int(max_depth), 
                                   min_samples_split=int(min_samples_split), 
                                   min_samples_leaf=int(min_samples_leaf),
                                   random_state=231)
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, SMOTE(random_state=42))
    
#     cv_results = cross_val_score(model, X_train_prepared, y_train, cv=10, scoring=profit_scoring)
    
#     return np.mean(cv_results)

In [44]:
params_random_forest = {
    'n_estimators': (25, 500),
    'max_depth': (1, 10),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20)
}

In [45]:
optimization = BayesianOptimization(evaluateRandomForest, params_random_forest, random_state=231)
optimization.maximize(n_iter=100)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        | -350.0    |  8.033    |  12.1     |  9.681    |  174.8    |
|  2        | -400.0    |  8.442    |  18.17    |  2.701    |  459.7    |
|  3        | -1.595e+0 |  1.545    |  4.434    |  3.301    |  242.9    |
|  4        | -385.0    |  7.375    |  10.18    |  2.458    |  56.78    |
|  5        | -505.0    |  4.833    |  5.166    |  16.58    |  91.53    |
|  6        | -350.0    |  8.072    |  12.14    |  9.72     |  174.4    |
|  7        | -350.0    |  8.119    |  12.21    |  9.76     |  174.6    |
|  8        | -420.0    |  7.975    |  12.12    |  9.544    |  174.6    |
|  9        | -400.0    |  7.645    |  18.01    |  2.919    |  459.5    |
|  10       | -350.0    |  8.869    |  12.36    |  9.958    |  174.0    |
|  11       | -420.0    |  7.71     |  12.31    |  9.926    |  174.9    |
|  12       | -350.0    |  9.089    | 

KeyboardInterrupt: 

In [37]:
optimization.max

{'target': 60.0,
 'params': {'max_depth': 7.588495835164709,
  'min_samples_leaf': 11.259814622879999,
  'min_samples_split': 3.981682006813868,
  'n_estimators': 55.181217441016784}}

### SGDClassifier

In [38]:
def evaluateSgd(alpha, l1_ratio, tol, penalty, loss):
    
    # 3 options, l1 by default
    penalty_str = 'l1'
    if int(penalty) == 0:
        penalty_str = 'l2'
    elif int(penalty) == 1:
        penalty_str = 'elasticnet'
    
    # 3 options, modified_huber by default
    loss_str = 'modified_huber'
    if int(loss) == 0:
        loss_str = 'hinge'
    elif int(loss) == 1:
        loss_str = 'log'
        
    
    model = SGDClassifier(alpha=alpha, l1_ratio=l1_ratio, tol=tol, penalty=penalty_str, loss=loss_str, random_state=231)
    
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, SMOTE(random_state=42))

#     cv_results = cross_val_score(model, X_train_prepared, y_train, cv=10, scoring=profit_scoring)
    
#     return np.mean(cv_results)

In [39]:
params_sgd = {
    'alpha': (1e-6, 1),
    'l1_ratio': (0, 1),
    'tol': (1e-9, 1e-1),
    'penalty': (0, 3),
    'loss': (0, 3)
}

In [46]:
optimization_sgd = BayesianOptimization(evaluateSgd, params_sgd, random_state=231)
optimization_sgd.maximize(n_iter=250)

|   iter    |  target   |   alpha   | l1_ratio  |   loss    |  penalty  |    tol    |
-------------------------------------------------------------------------------------
|  1        | -7.98e+03 |  0.7815   |  0.5842   |  1.28     |  0.946    |  0.08269  |
|  2        | -5.255e+0 |  0.9034   |  0.03894  |  2.746    |  0.1815   |  0.01807  |
|  3        | -855.0    |  0.0723   |  0.4588   |  2.125    |  1.45     |  0.002545 |
|  4        | -3.93e+03 |  0.0669   |  0.4259   |  0.6577   |  2.43     |  0.01401  |
|  5        | -6.58e+03 |  0.7512   |  0.2791   |  2.847    |  2.655    |  0.06102  |
|  6        | -5.655e+0 |  0.7949   |  0.1375   |  2.891    |  1.398    |  0.02334  |
|  7        | -6.905e+0 |  0.2285   |  0.659    |  1.156    |  2.647    |  0.03248  |
|  8        | -9.83e+03 |  0.262    |  0.1225   |  1.663    |  2.001    |  0.03901  |
|  9        | -7.03e+03 |  0.6054   |  0.8215   |  1.169    |  0.7796   |  0.04115  |
|  10       | -855.0    |  0.07219  |  0.459    |  2.1

In [47]:
optimization_sgd.max

{'target': 305.0,
 'params': {'alpha': 0.005854021438338096,
  'l1_ratio': 0.4547722688881384,
  'loss': 2.0803138889905366,
  'penalty': 1.4688569955709672,
  'tol': 0.02336999122867795}}

### SVM

In [73]:
from sklearn.svm import SVC

In [74]:
def evaluateSvm(C, gamma):
    
    model = SVC(C=C, gamma=gamma, random_state=231)
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, SMOTE(random_state=42))

In [75]:
params_svm = {
    'C': (1, 1000),
    'gamma': (0.001, 10)
}

In [76]:
optimization_svm = BayesianOptimization(evaluateSvm, params_svm, random_state=231)
optimization_svm.maximize(n_iter=250)

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
|  1        | -510.0    |  781.7    |  5.843    |
|  2        | -485.0    |  427.3    |  3.154    |
|  3        | -510.0    |  827.1    |  9.034    |
|  4        | -510.0    |  39.9     |  9.152    |
|  5        | -410.0    |  61.44    |  1.808    |
|  6        | -400.0    |  61.44    |  1.782    |
|  7        | -350.0    |  61.51    |  1.587    |
|  8        | -340.0    |  61.35    |  1.567    |
|  9        | -370.0    |  61.49    |  1.629    |
|  10       | -370.0    |  61.05    |  1.639    |
|  11       | -350.0    |  61.43    |  1.589    |
|  12       | -340.0    |  61.37    |  1.564    |
|  13       | -275.0    |  61.47    |  1.345    |
|  14       | -370.0    |  61.21    |  1.623    |
|  15       | -355.0    |  61.46    |  1.52     |
|  16       | -275.0    |  61.39    |  1.349    |
|  17       | -285.0    |  61.45    |  1.35     |
|  18       | -285.0    |  61.33    |  1.353    |


In [77]:
optimization_svm.max

{'target': -120.0,
 'params': {'C': 61.559671025178, 'gamma': 0.6412512564360546}}

### SVM - Poly kernel

In [78]:
def evaluateSvm_Poly(C, gamma, degree):
    
    model = SVC(C=C, gamma=gamma, degree=degree, kernel='poly', random_state=231)
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, SMOTE(random_state=42))

In [79]:
params_svm_poly = {
    'C': (1, 1000),
    'gamma': (0.001, 10),
    'degree': (1, 10)
}

In [33]:
optimization_svm = BayesianOptimization(evaluateSvm_Poly, params_svm_poly, random_state=231)
optimization_svm.maximize(n_iter=250, init_points=10)

NameError: name 'evaluateSvm_Poly' is not defined

In [77]:
optimization_svm.max

{'target': -120.0,
 'params': {'C': 61.559671025178, 'gamma': 0.6412512564360546}}

### XGBClassifier

In [38]:
def evaluateXgb(max_depth, n_estimators, gamma, reg_alpha, reg_lambda):
    
    model = XGBClassifier(max_depth=int(max_depth), n_estimators=int(n_estimators), gamma=gamma,
                          reg_alpha=reg_alpha, reg_lambda=reg_lambda, n_jobs=-1, random_state=231)
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, SMOTE(random_state=42))
    
#     cv_results = cross_val_score(model, X_train_prepared, y_train, cv=10, scoring=profit_scoring)
    
#     return np.mean(cv_results)

In [39]:
params_xgb = {
    'max_depth': (1, 10),
    'n_estimators': (25, 500),
    'gamma': (0, 1),
    'reg_alpha': (1e-9, 0.1),
    'reg_lambda': (1e-9, 0.1)
}

In [40]:
optimization_xgb = BayesianOptimization(evaluateXgb, params_xgb, random_state=231)
optimization_xgb.maximize(n_iter=100)

|   iter    |  target   |   gamma   | max_depth | n_esti... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
|  1        |  80.0     |  0.7815   |  6.258    |  227.7    |  0.03153  |  0.08269  |
|  2        | -45.0     |  0.9034   |  1.35     |  459.7    |  0.006051 |  0.01807  |
|  3        |  85.0     |  0.0723   |  5.129    |  361.5    |  0.04832  |  0.002545 |
|  4        |  90.0     |  0.0669   |  4.833    |  129.1    |  0.08101  |  0.01401  |
|  5        |  70.0     |  0.7512   |  3.512    |  475.8    |  0.0885   |  0.06102  |
|  6        |  105.0    |  0.7949   |  2.237    |  482.8    |  0.04659  |  0.02334  |
|  7        |  80.0     |  0.7867   |  2.412    |  482.8    |  0.04806  |  0.0264   |
|  8        |  130.0    |  0.7563   |  2.265    |  482.7    |  0.000928 |  0.02224  |
|  9        |  80.0     |  0.8811   |  2.218    |  482.9    |  0.09351  |  0.06362  |
|  10       |  80.0     |  0.8012   |  2.075    |  482

In [41]:
optimization_xgb.max

{'target': 160.0,
 'params': {'gamma': 0.3079585481450413,
  'max_depth': 3.1686809206091557,
  'n_estimators': 227.39776966615537,
  'reg_alpha': 0.0413745206438609,
  'reg_lambda': 0.031173917414878148}}

### Logistic Regression

In [57]:
from sklearn.linear_model import LogisticRegression

In [58]:
def evaluateLogReg(C):
    
#     if penalty < 0.5:
#         penalty = 'l1'
#     else:
#         penalty = 'l2'
    
    model = LogisticRegression(C=C, max_iter=1000, solver='lbfgs')
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, SMOTE(random_state=42))

In [59]:
params_logreg = {
#     'penalty': (0, 1),
    'C': (0.0001, 10000)
}

In [60]:
optimization_logreg = BayesianOptimization(evaluateLogReg, params_logreg, random_state=231)
optimization_logreg.maximize(n_iter=1000, init_points=20)

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  255.0    |  7.815e+0 |
|  2        |  255.0    |  5.842e+0 |
|  3        |  265.0    |  4.267e+0 |
|  4        |  240.0    |  3.153e+0 |
|  5        |  255.0    |  8.269e+0 |
|  6        |  255.0    |  9.034e+0 |
|  7        |  275.0    |  389.4    |
|  8        |  255.0    |  9.152e+0 |
|  9        |  250.0    |  605.1    |
|  10       |  225.0    |  1.807e+0 |
|  11       |  250.0    |  723.0    |
|  12       |  265.0    |  4.588e+0 |
|  13       |  255.0    |  7.084e+0 |
|  14       |  265.0    |  4.832e+0 |
|  15       |  275.0    |  254.5    |
|  16       |  250.0    |  669.0    |
|  17       |  265.0    |  4.259e+0 |
|  18       |  250.0    |  2.192e+0 |
|  19       |  255.0    |  8.101e+0 |
|  20       |  200.0    |  1.401e+0 |
|  21       |  275.0    |  326.7    |
|  22       |  275.0    |  364.7    |
|  23       |  275.0    |  283.6    |
|  24       |  275.0    |  379.2    |
|  25       

KeyboardInterrupt: 

In [61]:
275

275

### Stacking SGDClassifier and Logistic Regression

In [81]:
from imblearn.pipeline import make_pipeline

In [82]:
sgd = SGDClassifier(alpha=0.0026265889414660943, l1_ratio=0.40452166539879963, loss='modified_huber',
                    penalty='elasticnet', tol=0.09572138646802396, random_state=231)
log_reg = LogisticRegression(C=389.4, max_iter=1000, solver='lbfgs')

In [92]:
sgd_pipeline = make_pipeline(RandomOverSampler(random_state=42),
                             SGDClassifier(alpha=0.0026265889414660943, l1_ratio=0.40452166539879963, 
                                           loss='modified_huber', penalty='elasticnet', 
                                           tol=0.09572138646802396, random_state=231))

In [93]:
logreg_pipeline = make_pipeline(RandomOverSampler(random_state=42),
                                LogisticRegression(C=389.4, max_iter=1000, solver='lbfgs'))

In [94]:
sgd_pipeline.fit(X_train_prepared, y_train)

Pipeline(memory=None,
     steps=[('randomoversampler', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('sgdclassifier', SGDClassifier(alpha=0.0026265889414660943, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True...huffle=True, tol=0.09572138646802396,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [95]:
logreg_pipeline.fit(X_train_prepared, y_train)

Pipeline(memory=None,
     steps=[('randomoversampler', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('logisticregression', LogisticRegression(C=389.4, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [96]:
from mlens.ensemble import SuperLearner

In [97]:
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import RandomOverSampler

In [98]:
stacking = SuperLearner(scorer=profit_scoring, random_state=231, verbose=2)
stacking.add([sgd_pipeline, logreg_pipeline])
stacking.add_meta(DecisionTreeClassifier())

SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=1200, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex..._scorer))],
   n_jobs=-1, name='group-8', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=231, sample_size=20, scorer=make_scorer(profit_scorer),
       shuffle=False, verbose=2)

In [99]:
cross_val_imbalanced(stacking, X_train_prepared, y_train, RandomOverSampler(random_state=42))


Fitting 2 layers
Processing layer-1             

/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-1.0.2] Could not score pipeline-1. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-1.0.1] Could not score pipeline-1. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-2.0.2] Could not score pipeline-2. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-2.0.1] Could not score pipeline-2. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'

done | 00:00:00
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:00

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             

/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-1.0.1] Could not score pipeline-1. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-1.0.2] Could not score pipeline-1. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-2.0.2] Could not score pipeline-2. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-2.0.1] Could not score pipeline-2. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'

done | 00:00:00
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:00

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             

...

/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-1.0.1] Could not score pipeline-1. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-1.0.2] Could not score pipeline-1. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-2.0.1] Could not score pipeline-2. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-2.0.2] Could not score pipeline-2. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'

done | 00:00:00
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:00

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             

/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-1.0.2] Could not score pipeline-1. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-1.0.1] Could not score pipeline-1. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-2.0.1] Could not score pipeline-2. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [pipeline-2.0.2] Could not score pipeline-2. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'

done | 00:00:00
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:00

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00


265

### SGDClassifier - SMOTE Hyperparam Tuning

In [115]:
from imblearn.over_sampling import BorderlineSMOTE

In [119]:
def evaluateSgdSmote(k_neighbors, m_neighbors):
    
    model = SGDClassifier(alpha=0.0026265889414660943, l1_ratio=0.40452166539879963, loss='modified_huber',
                    penalty='elasticnet', tol=0.09572138646802396, random_state=231)
    
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, BorderlineSMOTE(
                                                                        k_neighbors=int(k_neighbors), 
                                                                        m_neighbors=int(m_neighbors),
                                                                        random_state=42))

#     cv_results = cross_val_score(model, X_train_prepared, y_train, cv=10, scoring=profit_scoring)
    
#     return np.mean(cv_results)

In [120]:
params_sgd_smote = {
    'k_neighbors': (2, 50),
    'm_neighbors': (2, 50)
}

In [121]:
optimization_sgd_smote = BayesianOptimization(evaluateSgdSmote, params_sgd_smote, random_state=42)
optimization_sgd_smote.maximize(n_iter=200, init_points=50)

|   iter    |  target   | k_neig... | m_neig... |
-------------------------------------------------
|  1        |  240.0    |  19.98    |  47.63    |
|  2        |  255.0    |  37.14    |  30.74    |
|  3        |  230.0    |  9.489    |  9.488    |
|  4        |  275.0    |  4.788    |  43.58    |
|  5        |  275.0    |  30.85    |  35.99    |
|  6        |  275.0    |  2.988    |  48.56    |
|  7        |  275.0    |  41.96    |  12.19    |
|  8        |  245.0    |  10.73    |  10.8     |
|  9        |  255.0    |  16.6     |  27.19    |
|  10       |  265.0    |  22.73    |  15.98    |
|  11       |  265.0    |  31.37    |  8.696    |
|  12       |  255.0    |  16.02    |  19.59    |
|  13       |  265.0    |  23.89    |  39.69    |
|  14       |  275.0    |  11.58    |  26.68    |
|  15       |  275.0    |  30.44    |  4.23     |
|  16       |  265.0    |  31.16    |  10.19    |
|  17       |  265.0    |  5.122    |  47.55    |
|  18       |  295.0    |  48.35    |  40.8     |


In [122]:
optimization_sgd_smote.max

{'target': 310.0,
 'params': {'k_neighbors': 3.650649013530483,
  'm_neighbors': 45.64737929978154}}